In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 30.3 MB/s 
     |████████████████████████████████| 182 kB 67.0 MB/s 
     |████████████████████████████████| 7.6 MB 48.4 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/MyDrive/Proyek/KPK

/content/drive/MyDrive/Proyek/KPK


In [4]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')

Downloading:   0%|          | 0.00/229k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

In [5]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

In [8]:
test.shape

(589, 3)

In [6]:
def sentence_to_vec(sentence):
  tokenized_text = tokenizer.tokenize(sentence)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  model = BertModel.from_pretrained('indobenchmark/indobert-base-p1',output_hidden_states = True)

  with torch.no_grad():
    outputs = model(tokens_tensor)
    hidden_states = outputs[2]

  token_vecs = hidden_states[-2][0]
  sentence_embedding = torch.mean(token_vecs, dim=0)
  return sentence_embedding

In [ ]:
similiarity = []
for ind in test.index:
    instansi_vec = sentence_to_vec(test['instansi'][ind])
    reference_vec = sentence_to_vec(test['reference'][ind])
    same = 1- cosine(instansi_vec, reference_vec)
    similiarity.append(same)

In [ ]:
test['similiarity'] = similiarity
test.to_csv('cosine_similiarity_result.csv',index=False)